In [1]:
!pip install openai-whisper==20231117
!pip install yt-dlp
!pip3 install whisper-timestamped

  Using cached triton-2.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torch-2.5.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torch-2.4.1-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-

In [6]:
import os
import json
import googleapiclient.discovery
import yt_dlp as youtube_dl
from urllib.parse import urlparse, parse_qs
from google.oauth2 import service_account
from google.cloud import storage
from io import BytesIO

import torch
import numpy as np
import pandas as pd
#extraer todos los archivos de audio .wav de una carpeta y transcribirlos
import glob
import re
import shutil  # Importar shutil para manejar operaciones de archivos

import whisper_timestamped as whisper
import uuid

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
model = whisper.load_model('medium', device=device)

100%|█████████████████████████████████████| 1.42G/1.42G [00:23<00:00, 64.0MiB/s]


In [7]:
# Ruta de trabajo
GLOBAL = "/content/"
AUDIO_DIR = os.path.join(GLOBAL, "audios")
TRANSCRIPT_DIR = os.path.join(GLOBAL, "transcriptions")

# Crear carpetas si no existen
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)


def download_audio(youtube_url, output_filename):
    output_path = os.path.join(AUDIO_DIR, output_filename)

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': output_path,
        'quiet': True,
        'no_warnings': True,
    }

    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        return output_path + ".wav"
    except Exception as e:
        print(f"[ERROR] Falló la descarga: {e}")
        return None

def clean_data(result):
    segments_df = pd.json_normalize(result['segments'])
    necessary_keys = {"start", "end", "text"}
    segments_df_cleaned = segments_df[list(necessary_keys)]
    segments_list = segments_df_cleaned.to_dict(orient='records')

    return {
        "segments": segments_list,
    }

def transcribe_audio(audio_path, video_id):
    print("Transcribiendo...")
    audio = whisper.load_audio(audio_path)
    result = whisper.transcribe(model=model, audio=audio, language='es')
    cleaned = clean_data(result)

    transcript_path = os.path.join(TRANSCRIPT_DIR, f"{video_id}.json")
    with open(transcript_path, "w") as f:
        json.dump(cleaned, f, indent=4, ensure_ascii=False)
    print(f"Transcripción guardada en: {transcript_path}")

    # Limpieza
    os.remove(audio_path)

def process_single_video(youtube_url):
    video_id = str(uuid.uuid4())  # ID único para el archivo
    print(f"Procesando video con ID: {video_id}")
    audio_path = download_audio(youtube_url, video_id)
    if audio_path:
        transcribe_audio(audio_path, video_id)

# Ejemplo de uso
process_single_video("https://www.youtube.com/watch?v=2ibqfxEAESo")

Procesando video con ID: a8fa00bb-f8a2-43f5-aef8-4d7ad6e394ad
Transcribiendo...


  4%|▎         | 5748/159410 [01:37<43:29, 58.89frames/s]


KeyboardInterrupt: 